<a href="https://colab.research.google.com/github/shazzad-hasan/practice-deep-learning-with-pytorch/blob/main/text_classification/spam_vs_ham.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# upload kaggle API key from your local machine
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shazzadraihan","key":"da63bbe0f8dcb3bd7fb35034046ca758"}'}

In [2]:
# make a kaggle dir, copy the API key to it
# and make sure the file in only readable by yourself (chmod 600)
!mkdir ~/.kaggle 
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
# use API command to download the dataset
!kaggle datasets download -d uciml/sms-spam-collection-dataset

  0% 0.00/211k [00:00<?, ?B/s]
100% 211k/211k [00:00<00:00, 86.6MB/s]


In [4]:
# uncompress the dataset
!unzip -qq sms-spam-collection-dataset.zip

In [5]:
!pip install torchtext==0.9.1
!pip install torch==1.8.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.1 MB 2.0 MB/s 
     |████████████████████████████████| 804.1 MB 2.7 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.8.1 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.8.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whee

In [6]:
# import required libraries
import torch
import torchtext
from torchtext.legacy import data

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [7]:
# check if cuda is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
  print("CUDA is not available")
else:
  print("CUDA is available")

device = torch.device('cuda') if train_on_gpu else torch.device('cpu')

CUDA is available


### Load and Visualize the data

In [8]:
# read data from text file
sms_df = pd.read_csv("/content/spam.csv", encoding="latin-1")

sms_df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


### Data pre-processing

In [9]:
# drop 3 unnamed columns
sms_df = sms_df.drop(columns = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
# rename the columns and v2
sms_df = sms_df.rename(index = str, columns = {"v1": "labels", "v2": "text"})

sms_df.head()

,labels,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [10]:
# split the dataset
train_data, test_data = train_test_split(sms_df, test_size = 0.2, random_state = 42)
# reset index
train_data.reset_index(drop=True)
test_data.reset_index(drop=True)

,labels,text
0,ham,"Funny fact Nobody teaches volcanoes 2 erupt, t..."
1,ham,I sent my scores to sophas and i had to do sec...
2,spam,We know someone who you know that fancies you....
3,ham,Only if you promise your getting out as SOON a...
4,spam,Congratulations ur awarded either å£500 of CD ...
...,...,...
1110,ham,&lt;DECIMAL&gt; m but its not a common car he...
1111,ham,Rightio. 11.48 it is then. Well arent we all u...
1112,ham,Yes i have. So that's why u texted. Pshew...mi...
1113,ham,"Get the door, I'm here"


In [11]:
print("Num of training data :", len(train_data))
print("Num of test data: ", len(test_data))

Num of training data : 4457
Num of test data:  1115


In [12]:
# save the training and test data to csv files
train_data.to_csv("train.csv", index=False)
test_data.to_csv("test.csv", index=False)

In [13]:
!ls

kaggle.json  sms-spam-collection-dataset.zip  test.csv
sample_data  spam.csv			      train.csv


In [14]:
import nltk
nltk.download("punkt")

from nltk import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [15]:
# specify how texts and labels should be processed
TEXT = data.Field(tokenize = word_tokenize)
LABEL = data.LabelField(dtype = torch.float)

In [16]:
# instantiate the filds objects
fields = [("labels", LABEL), ("text", TEXT)]

In [17]:
# specify row data for the data fields
train_data, test_data = data.dataset.TabularDataset.splits(path = '/content',
                                 train = "train.csv",
                                 test = "test.csv",
                                 format = "csv",
                                 skip_header = True,
                                 fields = fields)

In [18]:
print(f'Num of training data: {len(train_data)}')
print(f'Number of testing data: {len(test_data)}')

Num of training data: 4457
Number of testing data: 1115


In [19]:
train_data[0].__dict__.keys()

dict_keys(['labels', 'text'])

In [20]:
test_data[0].text

['Funny',
 'fact',
 'Nobody',
 'teaches',
 'volcanoes',
 '2',
 'erupt',
 ',',
 'tsunamis',
 '2',
 'arise',
 ',',
 'hurricanes',
 '2',
 'sway',
 'aroundn',
 'no',
 '1',
 'teaches',
 'hw',
 '2',
 'choose',
 'a',
 'wife',
 'Natural',
 'disasters',
 'just',
 'happens']

In [21]:
train_data[0].labels

'ham'

In [22]:
# print out all attributes associated with training
print(vars(train_data.examples[5]))

{'labels': 'ham', 'text': ['G', 'says', 'you', 'never', 'answer', 'your', 'texts', ',', 'confirm/deny']}


In [23]:
# limit the vocabulary in the training data
vocab_size = 10000

# numericalize 
TEXT.build_vocab(train_data, max_size = vocab_size)
LABEL.build_vocab(train_data)

In [24]:
print(f'Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}')
print(f'Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}')

Unique tokens in TEXT vocabulary: 10002
Unique tokens in LABEL vocabulary: 2


In [25]:
# most common 20 words
TEXT.vocab.freqs.most_common(20)

[('.', 3847),
 ('to', 1750),
 ('I', 1574),
 (',', 1468),
 ('you', 1462),
 ('?', 1256),
 ('!', 1134),
 ('a', 1068),
 ('the', 946),
 ('...', 923),
 ('&', 772),
 ('i', 760),
 ('and', 673),
 ('in', 663),
 ('is', 647),
 (';', 641),
 ('u', 636),
 ('me', 600),
 (':', 570),
 ('..', 544)]

In [26]:
# indices to tokens
TEXT.vocab.itos[:10]

['<unk>', '<pad>', '.', 'to', 'I', ',', 'you', '?', '!', 'a']

In [27]:
# numeric representation of individual string
print(LABEL.vocab.stoi)

defaultdict(None, {'ham': 0, 'spam': 1})


In [28]:
# define an iterator that batches examples of similar lengths together

batch_size = 64

train_iterator, test_iterator = data.BucketIterator.splits(
        (train_data, test_data),
        batch_size = batch_size,
        device = device,
        sort_key = lambda x: len(x.text),
        sort_within_batch = False)

### Define a Model

In [29]:
import torch.nn as nn
import torch.optim as optim

class RNN(nn.Module):
  def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
    super(RNN, self).__init__()

    self.embedding = nn.Embedding(input_dim, embedding_dim)
    self.rnn = nn.RNN(embedding_dim, hidden_dim)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, text):
    embedded = self.embedding(text)
    output, hidden = self.rnn(embedded)
    hidden_1D = hidden.squeeze(0)
    assert torch.equal(output[-1, :, :], hidden_1D)
    preds = self.fc(hidden_1D)
    return preds

class LSTM(nn.Module):
  def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, num_layers):
    super(LSTM, self).__init__()

    self.embedding = nn.Embedding(input_dim, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, text):
    embedded = self.embedding(text)
    output, (hidden,_) = self.lstm(embedded)
    hidden_1D = hidden.squeeze(0)
    assert torch.equal(output[-1, :, :], hidden_1D)
    preds = self.fc(hidden_1D)
    return preds

In [30]:
input_dim  = len(TEXT.vocab)
embedding_dim = 200
hidden_dim = 256
output_dim = 1
num_layers = 1

RNN_model = RNN(input_dim , embedding_dim, hidden_dim, output_dim)
RNN_model.to(device)

LSTM_model = LSTM(input_dim, embedding_dim, hidden_dim, output_dim, num_layers)
LSTM_model.to(device)

LSTM(
  (embedding): Embedding(10002, 200)
  (lstm): LSTM(200, 256)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)

### Train the model

In [31]:
def train(model, iterator, optimizer, criterion):  

  train_loss = 0.0

  model.train()
  for batch_idx, batch in enumerate(train_iterator):
    optimizer.zero_grad()
    predictions = model(batch.text).squeeze(1)
    loss = criterion(predictions, batch.labels)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()

  train_loss /= len(train_iterator)

  return train_loss

In [32]:
# --- Train RNN model -----

optimizer = optim.Adam(RNN_model.parameters(), lr = 0.01)
criterion = nn.BCEWithLogitsLoss()

num_epochs = 5
train_losses = []

for epoch in range(num_epochs):
  train_loss = train(RNN_model, train_iterator, optimizer, criterion)

  print('Epoch: {} | Training Loss: {:.2f}'.format(epoch, train_loss)) 

  train_losses.append(train_loss)

Epoch: 0 | Training Loss: 0.43
Epoch: 1 | Training Loss: 0.41
Epoch: 2 | Training Loss: 0.41
Epoch: 3 | Training Loss: 0.43
Epoch: 4 | Training Loss: 0.43


In [33]:
# --- Train LSTM model -----

optimizer = optim.Adam(LSTM_model.parameters(), lr = 0.01)
criterion = nn.BCEWithLogitsLoss()

num_epochs = 5
train_losses = []

for epoch in range(num_epochs):
  train_loss = train(LSTM_model, train_iterator, optimizer, criterion)

  print('Epoch: {} | Training Loss: {:.2f}'.format(epoch, train_loss)) 

  train_losses.append(train_loss)

Epoch: 0 | Training Loss: 0.42
Epoch: 1 | Training Loss: 0.40
Epoch: 2 | Training Loss: 0.40
Epoch: 3 | Training Loss: 0.40
Epoch: 4 | Training Loss: 0.40


### Test the model

In [34]:
def test(model, iterator):

  test_loss = 0.0

  model.eval()
  with torch.no_grad():
    for batch_idx, batch in enumerate(test_iterator):
      predictions = model(batch.text).squeeze(1)
      loss = criterion(predictions, batch.labels)
      rounded_preds = torch.round(torch.sigmoid(predictions))
      correct = (rounded_preds == batch.labels).float()
      test_loss += loss.item()

  test_loss /= len(test_iterator)

  return test_loss

In [35]:
# --- Test RNN model -----

test_loss = test(RNN_model, test_iterator)
print("Test Loss: {:.2f}".format(test_loss))

Test Loss: 0.46


In [36]:
# --- Test LSTM model -----

test_loss = test(LSTM_model, test_iterator)
print("Test Loss: {:.2f}".format(test_loss))

Test Loss: 0.38
